In [1]:
from sympy import *
import itertools as it
import numpy as np

In [2]:
# Make symbols: methods to dynamically create as many as needed strings and Sympy symbols
# Shift is starting index; default 0

def make_strings(N,s, shift = 0):
	my_strings = []
	for i in range(shift, N + shift):
	    tmp_st = f'{s}{i}'
	    my_strings.append(tmp_st)
	return my_strings

def make_symbols(N,s, shift = 0):
	my_symbols = []
	for i in range(shift, N + shift):
	    tmp_st = f'{s}{i}'
	    globals()[tmp_st] = Symbol(tmp_st)
	    my_symbols.append(globals()[tmp_st])
	return my_symbols

# Generate harmonic game

Given measure, find harmonic game

# Parameters

In [3]:
# Game skeleton

#numPlayers = 4

#skeleton = np.random.randint(2, 6, numPlayers)

skeleton = [2, 2, 2]
skeleton

[2, 2, 2]

In [4]:
# N
numPlayers = len(skeleton)
players = range(numPlayers)

# -------------------------
# Number of action profiles
# -------------------------

# A
numPures = prod(skeleton)

# List of A_{-i} = for each player, number of action profiles of other players
numPuresMinus = [  int(numPures / skeleton[i]) for i in players ]

# -------------------------
# Number of payoff degrees of freedom
# -------------------------

# AN; number of payoff degrees of freedom
numPays = numPlayers * numPures

# sum_i A_i; number of harmonic measure dofs
numMeas = sum(skeleton)

In [5]:
# Pure actions: list of N lists, each with Ai elements; pure actions of each player

#pures_play = [ make_strings(skeleton[i], f'a{i}', shift = 1) for i in players ]
pures_play = [ ['T','B'], ['L','R'], ['A','P'] ]
#pures_play = [ ['T','B'], ['L','R'] ]

In [6]:
pures_play

[['T', 'B'], ['L', 'R'], ['A', 'P']]

In [7]:
# Pure profiles; cartesian product of pure actions of each player
# Returns one list with A = numPures elements; each element is a tuple of strings

pures = list(it.product(*pures_play))

In [8]:
pures

[('T', 'L', 'A'),
 ('T', 'L', 'P'),
 ('T', 'R', 'A'),
 ('T', 'R', 'P'),
 ('B', 'L', 'A'),
 ('B', 'L', 'P'),
 ('B', 'R', 'A'),
 ('B', 'R', 'P')]

In [9]:
assert len(pures) == numPures

# Harmonic measure

In [10]:
# mu_values = [ np.random.randint(1, 5, skeleton[i]) for i in players]

#mu_values = [  [1, 0], [1, 2] ]

# mu_values = [  [2, 1], [3, 1], [4, 1] ]

# mu_values = [  [1, 2], [1, 3], [1,4], [1, 2, 3]  ]

mu_values = [  [1, 1], [1, 1], [1, 1] ]

# mu_values = [ make_symbols(skeleton[i], f'mu{i}', shift = 1) for i in players ]


# SYMBOLIC
# mu_values = [ symbols('mu_T mu_B'), symbols('mu_L mu_R'), symbols('mu_A mu_P') ]

mu_values

[[1, 1], [1, 1], [1, 1]]

In [11]:
# Pack mu in list of dicts

mu = [   dict(zip( pures_play[i], mu_values[i] )) for i in players    ]
mu

[{'T': 1, 'B': 1}, {'L': 1, 'R': 1}, {'A': 1, 'P': 1}]

In [12]:
numPures

8

In [13]:
numPuresMinus

[4, 4, 4]

In [14]:
numPays

24

In [15]:
numMeas

6

In [16]:
def print_payoff(payoff_dict, payoff_symbol):
    for i in players:
        for a in pures:
            print( f'{payoff_symbol}_{i}{a} = {payoff_dict[i][a]}' )
        print()

# Profiles of other players

In [17]:
# Pure profiles of other players
# Make list of N lists; the list pure_minus[i] contains the pure action profiles of players other than i
# Build taking the cartesian product of pure actions of all players other than i
# The size of the list pure_minus[i] is A_{-i} = \prod_{j \neq i} A_j

pures_minus = [ list( it.product( *( pures_play[:i] + pures_play[i+1:] ) ) ) for i in players ]

In [18]:
pures_minus

[[('L', 'A'), ('L', 'P'), ('R', 'A'), ('R', 'P')],
 [('T', 'A'), ('T', 'P'), ('B', 'A'), ('B', 'P')],
 [('T', 'L'), ('T', 'R'), ('B', 'L'), ('B', 'R')]]

In [19]:
numPuresMinus

[4, 4, 4]

In [20]:
for i in players:
    assert len(pures_minus[i]) == numPuresMinus[i]

# Make a

In [21]:
# Util to make (a_i, a_{-i}) given a_i and a_{-i} as tuple of strings (to be used as key for payoff dictionaries)
def make_pure(ai, a_minus_i, i):
    l = list(a_minus_i)
    return tuple(l[:i] + [ai] + l[i:])

In [22]:
def split_pure(a, i):
    
    ai = a[i]

    l = list(a)
    a_minus_i = tuple( l[:i] + l[i+1:] )
    return ai, a_minus_i

In [23]:
# check consistency
for i in players:
    for a in pures:
        ai, a_minus_i = split_pure(a, i)
        assert a == make_pure( ai, a_minus_i, i )

# Harmonic payoff

In [24]:
# harmonic payoff degrees of freedom; will build harmonic payoff h out of these

# to be determined harmonic payoff of player i (as many as A)

h_sym = [make_symbols(numPures, f'h{i}', shift = 1) for i in players]
h_sym

[[h01, h02, h03, h04, h05, h06, h07, h08],
 [h11, h12, h13, h14, h15, h16, h17, h18],
 [h21, h22, h23, h24, h25, h26, h27, h28]]

In [25]:
h = [  dict(zip(pures , h_sym[i] ))  for i in players]
h

[{('T', 'L', 'A'): h01,
  ('T', 'L', 'P'): h02,
  ('T', 'R', 'A'): h03,
  ('T', 'R', 'P'): h04,
  ('B', 'L', 'A'): h05,
  ('B', 'L', 'P'): h06,
  ('B', 'R', 'A'): h07,
  ('B', 'R', 'P'): h08},
 {('T', 'L', 'A'): h11,
  ('T', 'L', 'P'): h12,
  ('T', 'R', 'A'): h13,
  ('T', 'R', 'P'): h14,
  ('B', 'L', 'A'): h15,
  ('B', 'L', 'P'): h16,
  ('B', 'R', 'A'): h17,
  ('B', 'R', 'P'): h18},
 {('T', 'L', 'A'): h21,
  ('T', 'L', 'P'): h22,
  ('T', 'R', 'A'): h23,
  ('T', 'R', 'P'): h24,
  ('B', 'L', 'A'): h25,
  ('B', 'L', 'P'): h26,
  ('B', 'R', 'A'): h27,
  ('B', 'R', 'P'): h28}]

In [26]:
pures

[('T', 'L', 'A'),
 ('T', 'L', 'P'),
 ('T', 'R', 'A'),
 ('T', 'R', 'P'),
 ('B', 'L', 'A'),
 ('B', 'L', 'P'),
 ('B', 'R', 'A'),
 ('B', 'R', 'P')]

In [27]:
def is_unidev(a, b):
    if len(a) != len(b):
        return False

    differences = 0

    for ai, bi in zip(a, b):
        if ai != bi:
            differences += 1
            if differences > 1:
                return False

    return differences == 1

In [28]:
devs = [ ]
for i, a in enumerate(pures):
    for b in pures[i:]:
        if is_unidev(a,b):
            devs.append((a,b))

print(len(devs))
for d in devs:
    print(d)

12
(('T', 'L', 'A'), ('T', 'L', 'P'))
(('T', 'L', 'A'), ('T', 'R', 'A'))
(('T', 'L', 'A'), ('B', 'L', 'A'))
(('T', 'L', 'P'), ('T', 'R', 'P'))
(('T', 'L', 'P'), ('B', 'L', 'P'))
(('T', 'R', 'A'), ('T', 'R', 'P'))
(('T', 'R', 'A'), ('B', 'R', 'A'))
(('T', 'R', 'P'), ('B', 'R', 'P'))
(('B', 'L', 'A'), ('B', 'L', 'P'))
(('B', 'L', 'A'), ('B', 'R', 'A'))
(('B', 'L', 'P'), ('B', 'R', 'P'))
(('B', 'R', 'A'), ('B', 'R', 'P'))


In [29]:
def actor(d):
    assert d in devs
    a, b = d
    i = 0
    while a[i] == b[i]:
        i += 1
    return i

In [30]:
for d in devs:
    print(actor(d))

2
1
0
1
0
2
0
0
2
1
1
2


In [31]:
# deviazioni
dev_value = {}
for d in devs:
    a, b = d
    i = actor(d)
    dev_value[d] = h[i][a] - h[i][b]

In [32]:
dev_value

{(('T', 'L', 'A'), ('T', 'L', 'P')): h21 - h22,
 (('T', 'L', 'A'), ('T', 'R', 'A')): h11 - h13,
 (('T', 'L', 'A'), ('B', 'L', 'A')): h01 - h05,
 (('T', 'L', 'P'), ('T', 'R', 'P')): h12 - h14,
 (('T', 'L', 'P'), ('B', 'L', 'P')): h02 - h06,
 (('T', 'R', 'A'), ('T', 'R', 'P')): h23 - h24,
 (('T', 'R', 'A'), ('B', 'R', 'A')): h03 - h07,
 (('T', 'R', 'P'), ('B', 'R', 'P')): h04 - h08,
 (('B', 'L', 'A'), ('B', 'L', 'P')): h25 - h26,
 (('B', 'L', 'A'), ('B', 'R', 'A')): h15 - h17,
 (('B', 'L', 'P'), ('B', 'R', 'P')): h16 - h18,
 (('B', 'R', 'A'), ('B', 'R', 'P')): h27 - h28}

In [33]:
devs[0]

(('T', 'L', 'A'), ('T', 'L', 'P'))

In [34]:
dev_value[devs[0]]

h21 - h22

# System

In [35]:
pures

[('T', 'L', 'A'),
 ('T', 'L', 'P'),
 ('T', 'R', 'A'),
 ('T', 'R', 'P'),
 ('B', 'L', 'A'),
 ('B', 'L', 'P'),
 ('B', 'R', 'A'),
 ('B', 'R', 'P')]

In [36]:
pures_play

[['T', 'B'], ['L', 'R'], ['A', 'P']]

In [37]:
pures_minus

[[('L', 'A'), ('L', 'P'), ('R', 'A'), ('R', 'P')],
 [('T', 'A'), ('T', 'P'), ('B', 'A'), ('B', 'P')],
 [('T', 'L'), ('T', 'R'), ('B', 'L'), ('B', 'R')]]

In [38]:
# --------------------------------
# KEY: HARMONIC FUNCTION
# --------------------------------
def Hi(a, i):
    ai, a_minus_i = split_pure(a, i)
    return sum(   [   mu[i][bi] * ( h[i][a] - h[i][ make_pure(bi, a_minus_i, i) ] )    for bi in pures_play[i]   ]  )

In [39]:
for i in players:
    for bi in pures_play[i]:
        print(mu[i][bi] + 1)

2
2
2
2
2
2


In [40]:
for i in players:
    for a in pures:
        print( Hi( a, i ) )

h01 - h05
h02 - h06
h03 - h07
h04 - h08
-h01 + h05
-h02 + h06
-h03 + h07
-h04 + h08
h11 - h13
h12 - h14
-h11 + h13
-h12 + h14
h15 - h17
h16 - h18
-h15 + h17
-h16 + h18
h21 - h22
-h21 + h22
h23 - h24
-h23 + h24
h25 - h26
-h25 + h26
h27 - h28
-h27 + h28


In [41]:
# --------------------------------
# KEY: HARMONIC SYSTEM in degrees of freedom h, that is payoff to be determined, prescribed the harmonic measure
# --------------------------------
system = [  sum( [  Hi(a,i)   for i in players ] )   for a in pures  ]

In [42]:
system

[h01 - h05 + h11 - h13 + h21 - h22,
 h02 - h06 + h12 - h14 - h21 + h22,
 h03 - h07 - h11 + h13 + h23 - h24,
 h04 - h08 - h12 + h14 - h23 + h24,
 -h01 + h05 + h15 - h17 + h25 - h26,
 -h02 + h06 + h16 - h18 - h25 + h26,
 -h03 + h07 - h15 + h17 + h27 - h28,
 -h04 + h08 - h16 + h18 - h27 + h28]

In [43]:
# one equation per pure in pures
assert len(system) == numPures

In [44]:
# Unknowns of system to solve: the weights (one per player) + the non-strategic dofs
dofs = list(it.chain(*h_sym))
dofs

[h01,
 h02,
 h03,
 h04,
 h05,
 h06,
 h07,
 h08,
 h11,
 h12,
 h13,
 h14,
 h15,
 h16,
 h17,
 h18,
 h21,
 h22,
 h23,
 h24,
 h25,
 h26,
 h27,
 h28]

In [45]:
numDofs = len(dofs)

In [46]:
# Unknowns of system to solve: harmonic weights
assert numDofs == numPays

In [47]:
# Matrix of linear system
A, b = linear_eq_to_matrix(system, *dofs)

In [48]:
A

Matrix([
[ 1,  0,  0,  0, -1,  0,  0,  0,  1,  0, -1,  0,  0,  0,  0,  0,  1, -1,  0,  0,  0,  0,  0,  0],
[ 0,  1,  0,  0,  0, -1,  0,  0,  0,  1,  0, -1,  0,  0,  0,  0, -1,  1,  0,  0,  0,  0,  0,  0],
[ 0,  0,  1,  0,  0,  0, -1,  0, -1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  1, -1,  0,  0,  0,  0],
[ 0,  0,  0,  1,  0,  0,  0, -1,  0, -1,  0,  1,  0,  0,  0,  0,  0,  0, -1,  1,  0,  0,  0,  0],
[-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  1,  0, -1,  0,  0,  0,  0,  0,  1, -1,  0,  0],
[ 0, -1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  1,  0, -1,  0,  0,  0,  0, -1,  1,  0,  0],
[ 0,  0, -1,  0,  0,  0,  1,  0,  0,  0,  0,  0, -1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  1, -1],
[ 0,  0,  0, -1,  0,  0,  0,  1,  0,  0,  0,  0,  0, -1,  0,  1,  0,  0,  0,  0,  0,  0, -1,  1]])

In [49]:
sol = solve(system, dofs)

In [50]:
# Solution
sol

{h01: h05 + h15 - h17 + h25 - h26,
 h02: h06 + h16 - h18 - h25 + h26,
 h03: h07 - h15 + h17 + h27 - h28,
 h04: h08 - h16 + h18 - h27 + h28,
 h11: h13 - h15 + h17 + h23 - h24 + h27 - h28,
 h12: h14 - h16 + h18 - h23 + h24 - h27 + h28,
 h21: h22 - h23 + h24 - h25 + h26 - h27 + h28}

In [51]:
# Replace sol in dofs

extracted_sol = [e.subs(sol) for e in dofs]

In [52]:
extracted_sol

[h05 + h15 - h17 + h25 - h26,
 h06 + h16 - h18 - h25 + h26,
 h07 - h15 + h17 + h27 - h28,
 h08 - h16 + h18 - h27 + h28,
 h05,
 h06,
 h07,
 h08,
 h13 - h15 + h17 + h23 - h24 + h27 - h28,
 h14 - h16 + h18 - h23 + h24 - h27 + h28,
 h13,
 h14,
 h15,
 h16,
 h17,
 h18,
 h22 - h23 + h24 - h25 + h26 - h27 + h28,
 h22,
 h23,
 h24,
 h25,
 h26,
 h27,
 h28]

# General solution here

In [53]:
# zip solution in dictionary with dofs
sol_dict = dict(zip(dofs, extracted_sol))

In [54]:
sol_dict

{h01: h05 + h15 - h17 + h25 - h26,
 h02: h06 + h16 - h18 - h25 + h26,
 h03: h07 - h15 + h17 + h27 - h28,
 h04: h08 - h16 + h18 - h27 + h28,
 h05: h05,
 h06: h06,
 h07: h07,
 h08: h08,
 h11: h13 - h15 + h17 + h23 - h24 + h27 - h28,
 h12: h14 - h16 + h18 - h23 + h24 - h27 + h28,
 h13: h13,
 h14: h14,
 h15: h15,
 h16: h16,
 h17: h17,
 h18: h18,
 h21: h22 - h23 + h24 - h25 + h26 - h27 + h28,
 h22: h22,
 h23: h23,
 h24: h24,
 h25: h25,
 h26: h26,
 h27: h27,
 h28: h28}

In [55]:
dev_value

{(('T', 'L', 'A'), ('T', 'L', 'P')): h21 - h22,
 (('T', 'L', 'A'), ('T', 'R', 'A')): h11 - h13,
 (('T', 'L', 'A'), ('B', 'L', 'A')): h01 - h05,
 (('T', 'L', 'P'), ('T', 'R', 'P')): h12 - h14,
 (('T', 'L', 'P'), ('B', 'L', 'P')): h02 - h06,
 (('T', 'R', 'A'), ('T', 'R', 'P')): h23 - h24,
 (('T', 'R', 'A'), ('B', 'R', 'A')): h03 - h07,
 (('T', 'R', 'P'), ('B', 'R', 'P')): h04 - h08,
 (('B', 'L', 'A'), ('B', 'L', 'P')): h25 - h26,
 (('B', 'L', 'A'), ('B', 'R', 'A')): h15 - h17,
 (('B', 'L', 'P'), ('B', 'R', 'P')): h16 - h18,
 (('B', 'R', 'A'), ('B', 'R', 'P')): h27 - h28}

In [56]:
sol_dev_value = { d : dev_value[d].subs(sol_dict) for d in dev_value }

def print_dict(my_dict):
    for d in my_dict:
        print(d, ":", my_dict[d])
        print()
print_dict(sol_dev_value)

(('T', 'L', 'A'), ('T', 'L', 'P')) : -h23 + h24 - h25 + h26 - h27 + h28

(('T', 'L', 'A'), ('T', 'R', 'A')) : -h15 + h17 + h23 - h24 + h27 - h28

(('T', 'L', 'A'), ('B', 'L', 'A')) : h15 - h17 + h25 - h26

(('T', 'L', 'P'), ('T', 'R', 'P')) : -h16 + h18 - h23 + h24 - h27 + h28

(('T', 'L', 'P'), ('B', 'L', 'P')) : h16 - h18 - h25 + h26

(('T', 'R', 'A'), ('T', 'R', 'P')) : h23 - h24

(('T', 'R', 'A'), ('B', 'R', 'A')) : -h15 + h17 + h27 - h28

(('T', 'R', 'P'), ('B', 'R', 'P')) : -h16 + h18 - h27 + h28

(('B', 'L', 'A'), ('B', 'L', 'P')) : h25 - h26

(('B', 'L', 'A'), ('B', 'R', 'A')) : h15 - h17

(('B', 'L', 'P'), ('B', 'R', 'P')) : h16 - h18

(('B', 'R', 'A'), ('B', 'R', 'P')) : h27 - h28



In [57]:
devs

[(('T', 'L', 'A'), ('T', 'L', 'P')),
 (('T', 'L', 'A'), ('T', 'R', 'A')),
 (('T', 'L', 'A'), ('B', 'L', 'A')),
 (('T', 'L', 'P'), ('T', 'R', 'P')),
 (('T', 'L', 'P'), ('B', 'L', 'P')),
 (('T', 'R', 'A'), ('T', 'R', 'P')),
 (('T', 'R', 'A'), ('B', 'R', 'A')),
 (('T', 'R', 'P'), ('B', 'R', 'P')),
 (('B', 'L', 'A'), ('B', 'L', 'P')),
 (('B', 'L', 'A'), ('B', 'R', 'A')),
 (('B', 'L', 'P'), ('B', 'R', 'P')),
 (('B', 'R', 'A'), ('B', 'R', 'P'))]

In [58]:
# impose CLUB face
vertical_devs = [d for d in devs if actor(d) == 0]
vertical_devs

[(('T', 'L', 'A'), ('B', 'L', 'A')),
 (('T', 'L', 'P'), ('B', 'L', 'P')),
 (('T', 'R', 'A'), ('B', 'R', 'A')),
 (('T', 'R', 'P'), ('B', 'R', 'P'))]

In [59]:
vertical_devs_values = { d:sol_dev_value[d] for d in vertical_devs }
print_dict(vertical_devs_values)

(('T', 'L', 'A'), ('B', 'L', 'A')) : h15 - h17 + h25 - h26

(('T', 'L', 'P'), ('B', 'L', 'P')) : h16 - h18 - h25 + h26

(('T', 'R', 'A'), ('B', 'R', 'A')) : -h15 + h17 + h27 - h28

(('T', 'R', 'P'), ('B', 'R', 'P')) : -h16 + h18 - h27 + h28



In [60]:
simplify(sum(vertical_devs_values.values()))

0

In [61]:
mu

[{'T': 1, 'B': 1}, {'L': 1, 'R': 1}, {'A': 1, 'P': 1}]

---

# Build solution instance

In [62]:
# fix remaining dofs; generate dictionary with dofs keys and random values
fix_remaining_dofs = dict(zip(dofs, np.random.randint(-5, 5, numDofs)))

In [63]:
fix_remaining_dofs

{h01: 3,
 h02: 2,
 h03: -2,
 h04: 4,
 h05: -3,
 h06: -5,
 h07: -5,
 h08: 2,
 h11: -1,
 h12: 0,
 h13: 0,
 h14: 3,
 h15: 1,
 h16: 3,
 h17: 4,
 h18: 2,
 h21: -2,
 h22: -2,
 h23: 1,
 h24: 0,
 h25: 4,
 h26: -1,
 h27: 0,
 h28: 2}

In [64]:
# fix remaining dofs to random number by subbing in sol_dict
sol_dict_instance = { key : sol_dict[key].subs(fix_remaining_dofs) for key in sol_dict}

In [65]:
sol_dict_instance

{h01: -1,
 h02: -9,
 h03: -4,
 h04: 3,
 h05: -3,
 h06: -5,
 h07: -5,
 h08: 2,
 h11: 2,
 h12: 3,
 h13: 0,
 h14: 3,
 h15: 1,
 h16: 3,
 h17: 4,
 h18: 2,
 h21: -6,
 h22: -2,
 h23: 1,
 h24: 0,
 h25: 4,
 h26: -1,
 h27: 0,
 h28: 2}

In [66]:
h

[{('T', 'L', 'A'): h01,
  ('T', 'L', 'P'): h02,
  ('T', 'R', 'A'): h03,
  ('T', 'R', 'P'): h04,
  ('B', 'L', 'A'): h05,
  ('B', 'L', 'P'): h06,
  ('B', 'R', 'A'): h07,
  ('B', 'R', 'P'): h08},
 {('T', 'L', 'A'): h11,
  ('T', 'L', 'P'): h12,
  ('T', 'R', 'A'): h13,
  ('T', 'R', 'P'): h14,
  ('B', 'L', 'A'): h15,
  ('B', 'L', 'P'): h16,
  ('B', 'R', 'A'): h17,
  ('B', 'R', 'P'): h18},
 {('T', 'L', 'A'): h21,
  ('T', 'L', 'P'): h22,
  ('T', 'R', 'A'): h23,
  ('T', 'R', 'P'): h24,
  ('B', 'L', 'A'): h25,
  ('B', 'L', 'P'): h26,
  ('B', 'R', 'A'): h27,
  ('B', 'R', 'P'): h28}]

In [67]:
for D in h:
    for key in D:
        print(key, ":", D[key].subs(sol_dict_instance))
    print()

('T', 'L', 'A') : -1
('T', 'L', 'P') : -9
('T', 'R', 'A') : -4
('T', 'R', 'P') : 3
('B', 'L', 'A') : -3
('B', 'L', 'P') : -5
('B', 'R', 'A') : -5
('B', 'R', 'P') : 2

('T', 'L', 'A') : 2
('T', 'L', 'P') : 3
('T', 'R', 'A') : 0
('T', 'R', 'P') : 3
('B', 'L', 'A') : 1
('B', 'L', 'P') : 3
('B', 'R', 'A') : 4
('B', 'R', 'P') : 2

('T', 'L', 'A') : -6
('T', 'L', 'P') : -2
('T', 'R', 'A') : 1
('T', 'R', 'P') : 0
('B', 'L', 'A') : 4
('B', 'L', 'P') : -1
('B', 'R', 'A') : 0
('B', 'R', 'P') : 2



# Result: harmonic game

In [68]:
print(skeleton)

[2, 2, 2]


In [69]:
sol_dict_instance.values()

dict_values([-1, -9, -4, 3, -3, -5, -5, 2, 2, 3, 0, 3, 1, 3, 4, 2, -6, -2, 1, 0, 4, -1, 0, 2])

# End